In [ ]:
# Tạo hàm lấy danh sách các tệp , các thư mục con
def get_files_of_type(parent_path:str, filetype:str, as_dict:bool=False) :

    assert os.path.isdir(parent_path), f'{parent_path} is not a valid directory.'

    path_list = sorted([os.path.join(root,file)
                        for root,_,files in os.walk(parent_path)
                        for file in files if file.endswith(filetype)])

    assert len(path_list) > 0, f'{parent_path} contains 0 files of file type {filetype}.'
    
    if as_dict :
        
        path_dict = {".".join(os.path.split(path)[-1].split(".")[:-1]):path for path in path_list}

        return path_dict

    return path_list
    
# chuyển đổi các cột có chứa từ 'time' trong tên cột sang kiểu dữ liệu datetime
def dataframe_datetime(df) :
    
    for c in [c for c in df.columns if 'time' in c] :

        try :
            df[c] = pd.to_datetime(df[c])
        except Exception :
            pass

    return df

In [ ]:
import pandas as pd
import gc


icustays_df = pd.read_csv('/kaggle/input/data-icu/icustays.csv',low_memory=True)

admissions_df = pd.read_csv('/kaggle/input/data-icu/admissions.csv',low_memory=True)


icustays_df = dataframe_datetime(icustays_df)
admissions_df = dataframe_datetime(admissions_df)

# Kiểm tra xem tất cả hadm_ids cho bảng icustays_df có nằm trong bảng admissions
assert set(icustays_df['hadm_id']).issubset(set(admissions_df['hadm_id']))

In [ ]:
icustays_df.head()

### icustays_df
subject_id: Mã định danh (ID) của bệnh nhân

hadm_id: Mã định danh của lần nhập viện (Hospital Admission ID).

stay_id: Mã định danh của lần lưu trú tại bệnh viện.

first_careunit: Đơn vị chăm sóc đầu tiên nơi bệnh nhân được đưa vào

last_careunit: Đơn vị chăm sóc cuối cùng mà bệnh nhân được chuyển đến trước khi ra viện.

intime: Thời gian mà bệnh nhân nhập viện hoặc nhập đơn vị chăm sóc

outtime: Thời gian mà bệnh nhân xuất viện hoặc xuất khỏi đơn vị chăm sóc

los: Thời gian lưu trú (Length of Stay).

age: Tuổi của bệnh nhân khi nhập viện.

gender: Giới tính của bệnh nhân (nam hoặc nữ).

insurance: Loại bảo hiểm y tế của bệnh nhân

admission location: Nơi mà bệnh nhân được nhập viện từ đó, như từ nhà, từ cơ sở y tế khác, hoặc từ phòng cấp cứu.

race: Chủng tộc của bệnh nhân ủa bệnh nhân.

In [ ]:
admissions_df

### Admission_df
subject_id: Mã định danh của bệnh nhân

hadm_id: Mã định danh của lần nhập viện (Hospital Admission ID)

admittime: Thời gian nhập viện - ngày giờ mà bệnh nhân chính thức nhập viện.

dischtime: Thời gian xuất viện

deathtime: Thời gian tử vong của bệnh nhân

admission_type: Loại nhập viện, mô tả lý do hoặc hình thức nhập viện (ví dụ: cấp cứu, theo kế hoạch, chấn thương, chuyển viện, v.v.).

admission_location: Nơi xuất phát của bệnh nhân trước khi nhập viện

discharge_location: Nơi bệnh nhân được xuất viện hoặc chuyển đi sau khi rời bệnh viện

insurance: Loại bảo hiểm y tế của bệnh nhân

language: Ngôn ngữ chính của bệnh nhân

marital_status: Tình trạng hôn nhân của bệnh nhân

race: Chủng tộc của bệnh nhân

edregtime: Thời gian đăng ký tại khoa cấp cứu (Emergency Department Registration Time)

edouttime: Thời gian rời khỏi khoa cấp cứu (Emergency Department Out Time)

hospital_expire_flag: đánh dấu việc bệnh nhân có qua đời trong bệnh viện hay không. Giá trị là "1" nếu bệnh nhân qua đời trong bệnh viện và "0" nếu không

In [ ]:
import numpy as np 

# Giữ lại những lần lưu trú có thời gian lưu trú lớn hơn hoặc bằng 1 ngày (lấy từ admissions_df)
icustays_df = icustays_df[icustays_df['los'] >= 1]

icu_hadm_ids = set(icustays_df['hadm_id']) & set(admissions_df['hadm_id'])

admissions_df = admissions_df[admissions_df['hadm_id'].isin(icu_hadm_ids)]
subjects = set(admissions_df['subject_id'])

stays = set(icustays_df['stay_id'])


In [ ]:
# Lấy tuổi của bệnh nhân (dựa trên quy định của HIPAA) và giới tính - giới tính, 1 = nữ, 0 = nam
patients_df = pd.read_csv('/kaggle/input/data-icu/patients.csv')

patients_df = patients_df[patients_df['subject_id'].isin(subjects)]

# đổi thành dạng số nhị phân (0 hoặc 1)
patients_df['gender'] = np.array(patients_df['gender'] == 'F').astype(int)

anchor_age_tuples = patients_df.apply(lambda row : (row['subject_id']
                                                    , row['anchor_age'], row['anchor_year']), 1)

anchor_age_dict = {subject_id:{'anchor_age':anchor_age, 'anchor_year':anchor_year} 
                   for subject_id,anchor_age,anchor_year in anchor_age_tuples}

gender_dict = dict(zip(patients_df['subject_id'], patients_df['gender']))

icustays_df['age'] = icustays_df.apply(
    lambda row : anchor_age_dict[row['subject_id']]['anchor_age'] + (row['intime'].year - anchor_age_dict[row['subject_id']]['anchor_year']), 1)

icustays_df['age'] = icustays_df.apply(lambda row : row['age'] if row['age'] <= 90 else 90, 1) # định nghĩa luật HIPAA 

icustays_df['gender'] = icustays_df.apply(lambda row: gender_dict[row['subject_id']], 1)

In [ ]:
patients_df

### patients_df
subject_id: Mã định danh của bệnh nhân

gender: Giới tính của bệnh nhân

anchor_age: Tuổi của bệnh nhân tại thời điểm được ghi nhận (anchor)

anchor_year: Năm của sự kiện "neo" (anchor year), thường là năm xảy ra sự kiện quan trọng mà các thông tin khác được liên kết hoặc ghi lại, như năm nhập viện hoặc thời điểm theo dõi bệnh nhân.

anchor_year_group: Nhóm năm của sự kiện "neo". Đây là một trường có thể chia các bệnh nhân thành các nhóm dựa trên năm của sự kiện "neo"

dod (Date of Death): Ngày tử vong của bệnh nhân. có).

In [ ]:
# Thêm bảo hiểm('insurance') và địa điểm nhập học(admission location) vào mỗi subject_id trong icustays_df
admissions_df = admissions_df[admissions_df['subject_id'].isin(subjects)]

# tạo từ điển insurance_dict
insurance_dict = dict(zip(admissions_df['subject_id'], admissions_df['insurance']))

icustays_df['insurance'] = icustays_df['subject_id'].map(insurance_dict)

admloc_dict = dict(zip(admissions_df['subject_id'], admissions_df['admission_location']))

icustays_df['admission location'] = icustays_df['subject_id'].map(admloc_dict)

race_dict = dict(zip(admissions_df['subject_id'], admissions_df['race']))

icustays_df['race'] = icustays_df['subject_id'].map(race_dict)

In [ ]:
# Xóa các tuple không còn cần thiết nữa
del anchor_age_tuples
del anchor_age_dict
del gender_dict

In [ ]:
import os


chartevent_definitions = pd.read_csv('/kaggle/input/data-icu/d_items.csv',low_memory=True)

chartevent_definitions = chartevent_definitions[(chartevent_definitions['linksto'] == 'chartevents') & (chartevent_definitions['category'] == 'Routine Vital Signs')]

routine_vital_items = chartevent_definitions['itemid'].values

print(f'Number of unique routine vital sign item ids: {len(routine_vital_items)}')

chartevents = pd.read_csv('/kaggle/input/data-icu/chartevents.csv', chunksize = 10000000)

stays = set(icustays_df['stay_id'])

subjects = set(icustays_df['subject_id'])

# Tạo hai đường dẫn: ./data và ./data/mimic_chartevents
for p in [os.path.join('.', 'data'), os.path.join('.', 'data', 'mimic_chartevents')] :
    if not os.path.isdir(p) :
        os.mkdir(p)

mimic_chartevents_parent = os.path.join('.', 'data', 'mimic_chartevents')

if len(os.listdir(mimic_chartevents_parent)) == 0 : 

    for i,chunk in enumerate(chartevents) : 
        
        original_size = chunk.shape
        chunk = chunk[(chunk['stay_id'].isin(stays)) & (chunk['itemid'].isin(routine_vital_items))]
        chunk.to_csv(os.path.join(mimic_chartevents_parent, f'mimic_vitals_{i}.csv'), index = False)
        print(f'Chunk {i}: selected {chunk.shape[0]} from {original_size[0]} rows.')

chartevents_df = pd.concat([pd.read_csv(path) for path in get_files_of_type(mimic_chartevents_parent, filetype = 'csv')], axis=0, ignore_index=True)
chartevents_df.reset_index(drop = True, inplace = True)
chartevents_df['charttime'] = pd.to_datetime(chartevents_df['charttime'])


#  Nhóm chartevents_df theo 'stay_id' và 'itemid', và giữ hàng đầu tiên cho mỗi nhóm - dấu hiệu sinh tồn đầu tiên cho mỗi bệnh nhân
filtered_chartevents_df = chartevents_df.groupby(['stay_id', 'itemid']).first().reset_index()
print(f'{filtered_chartevents_df.shape=}')

In [ ]:
chartevent_definitions.head()

### chartevent_definitions
itemid: Mã định danh để nhận diện các thông số xét nghiệm hoặc đo lường.

label:Tên đầy đủ hoặc mô tả của mục

abbreviation: Viết tắt của tên mục

category: Phân loại nhóm các xét nghiệm, chỉ số hay thông số y tế

unitname: Đơn vị đo lường cho mục. Ví dụ: "mg/dL" cho nồng độ chất trong máu, "mmHg" cho huyết áp, hoặc "bpm" cho nhịp tim.

param_type: Loại thông số (parameter type) - thông số định lượng (số lượng) hay định tính (chất lượng), hoặc có thể là một thông số sinh lý.

lownormalvalue:Giá trị thấp bình thường cho thông số.

In [ ]:
# 1.kết hợp hai DataFrame dựa trên các cột chung
filtered_chartevents_df = pd.merge(
    filtered_chartevents_df,
    icustays_df,
    how='inner', # chỉ giữ lại các hàng mà có giá trị chung 
    
    on=['stay_id','subject_id','hadm_id']
    #chỉ định các cột sẽ được sử dụng để thực hiện phép hợp nhất
)

In [ ]:
filtered_chartevents_df.info()

In [ ]:
filtered_chartevents_df.describe()

##  1.Phân phối độ tuổi của bệnh nhân

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#  1.Phân phối độ tuổi của bệnh nhân
plt.figure(figsize=(12, 6))
sns.histplot(filtered_chartevents_df['age'], bins=30, kde=True)
plt.title('Phân phối độ tuổi của bệnh nhân', fontsize = 16)
plt.xlabel('Tuổi', fontsize = 14)
plt.ylabel('Tần suất', fontsize = 14)
plt.show()

## 2.Phân phối giới tính

In [ ]:
# 2.Phân phối giới tính
plt.figure(figsize=(8, 6))
sns.countplot(data=filtered_chartevents_df, x='gender', order=[0, 1], 
              label=['Nam', 'Nữ'], color='lightblue')
plt.title('Phân phối giới tính của bệnh nhân')
plt.xlabel('Giới tính')
plt.ylabel('Số lượng')
plt.xticks(ticks=[0, 1], labels=['Nam', 'Nữ'])
plt.show()

In [ ]:
import pandas as pd
labs_df = pd.read_csv('/kaggle/input/data-icu/labevents.csv',low_memory=True)
labs_df = labs_df[labs_df['subject_id'].isin(subjects)]
labs_df = dataframe_datetime(labs_df)
print(f'{labs_df.shape=}')

In [ ]:
labs_df.head()

### labs_df
labevent_id: Mã định danh duy nhất cho mỗi sự kiện xét nghiệm (labevent)

subject_id:Mã định danh của bệnh nhân

hadm_id:Mã định danh của lần nhập viện
specimen_id: Mã định danh của mẫu xét nghiệm
itemid:Mã định danh duy nhất cho từng loại xét nghiệm hoặc chỉ số đo lường. charttime:Thời gian mà xét nghiệm được thực hiện hoặc ghi lại,
storetime:Thời gian mà kết quả xét nghiệm được lưu trữ trong hệ thống
value:Giá trị xét nghiệm dưới dạng văn bản valuenum:Giá trị số của kết quả xét nghiệm
valueuom:Đơn vị đo lường cho giá trị của kết quả xét nghiệm (ví dụ: mg/dL, bpm).
ref_range_lower:Giá trị thấp nhất trong khoảng bình thường cho xét nghiệm
ref_range_upper:Giá trị cao nhất trong khoảng bình thường cho xét nghiệm
flag:Cờ cảnh báo hoặc nhắc nhở về kết quả xét nghiệm
priority: Độ ưu tiên của xét nghiệm
comments:Các chú thích hoặc ghi chú bổ sung về xét nghiệm hoặc kết quả


In [ ]:
#  kết hợp hai DataFrame dựa trên các cột chung
filtered_labs_df = pd.merge(
    labs_df,
    icustays_df,
    how='inner',
    left_on=['subject_id', 'hadm_id'],
    right_on=['subject_id', 'hadm_id']
)

filtered_labs_df['charttime'] = pd.to_datetime(filtered_labs_df['charttime'])

# lọc các hàng có charttime nằm trong khoảng thời gian từ intime đến outtime
condition = (filtered_labs_df['charttime'] >= filtered_labs_df['intime']) & (filtered_labs_df['charttime'] <= filtered_labs_df['outtime'])

# chỉ định rằng các hàng thỏa mãn điều kiện sẽ có stay_id được cập nhật
filtered_labs_df.loc[condition, 'stay_id'] = filtered_labs_df.loc[condition, 'stay_id']

# Display the resulting DataFrame
print(filtered_labs_df[['subject_id','hadm_id','stay_id','itemid','valuenum']])
print(f'{filtered_labs_df.shape=}')

In [ ]:
filtered_labs_df.head()

In [ ]:
filtered_labs_df.info()

In [ ]:
filtered_labs_df.describe()

In [ ]:
# Làm sạch và chuyển đổi cột 'giá trị' thành số
filtered_labs_df['value'] = pd.to_numeric(filtered_labs_df['value'], errors='coerce')

# Kiểm tra trùng lặp trong cot 'stay_id', 'itemid'
duplicates_for_stay_id = filtered_labs_df[filtered_labs_df.duplicated(subset=['stay_id', 'itemid'], keep=False)]

# Tính giá trị trung bình cho itemid, stay_id trùng lặp trong mỗi nhóm stay_id
averages_for_duplicates = duplicates_for_stay_id.groupby(['stay_id', 'itemid'])['value'].mean().reset_index()

# Gộp các giá trị trung bình đã tính toán trở lại DataFrame gốc
filtered_labs_df = pd.merge(filtered_labs_df, averages_for_duplicates, how='left', on=['stay_id', 'itemid'], suffixes=('', '_avg'))

# Điền giá trị NaN vào cột 'giá trị' ban đầu bằng giá trị trung bình đã tính toán
filtered_labs_df['value'].fillna(filtered_labs_df['value_avg'], inplace=True)

# Xóa cột 'value_avg' thừa
filtered_labs_df.drop(columns=['value_avg'], inplace=True)

In [ ]:
import pandas as pd
pharm_df = pd.read_csv('/kaggle/input/data-icu/prescriptions.csv',low_memory=True)


filtered_pharm_df = pd.merge(
    pharm_df,
    icustays_df,
    how='inner',
    left_on=['subject_id', 'hadm_id'],
    right_on=['subject_id', 'hadm_id']
)

filtered_pharm_df['starttime'] = pd.to_datetime(filtered_pharm_df['starttime'])


condition = (filtered_pharm_df['starttime'] >= filtered_pharm_df['intime']) & (filtered_pharm_df['starttime'] <= filtered_pharm_df['outtime'])
filtered_pharm_df.loc[condition, 'stay_id'] = filtered_pharm_df.loc[condition, 'stay_id']


print(filtered_pharm_df[['subject_id','hadm_id','stay_id','drug']])
print(f'{filtered_pharm_df.shape=}')

In [ ]:
filtered_pharm_df.head()

In [ ]:
filtered_pharm_df.describe()

In [ ]:
filtered_pharm_df.info()

In [ ]:
# Tạo khung dữ liệu để phân tích
analysis_chartevents_df = filtered_chartevents_df[['stay_id', 'subject_id', 'itemid', 'valuenum','insurance','gender','age','race']]
analysis_labs_df = filtered_labs_df[['subject_id', 'stay_id', 'itemid', 'value','insurance','gender','age','race']]
analysis_pharm_df = filtered_pharm_df[['subject_id', 'stay_id', 'drug','insurance','gender','age','race']]

analysis_chartevents_df

In [ ]:
analysis_chartevents_df.info

In [ ]:
analysis_chartevents_df.describe()

In [ ]:
analysis_labs_df.head()

In [ ]:
analysis_labs_df.info()

In [ ]:
analysis_labs_df.describe()

In [ ]:
analysis_pharm_df.head()

In [ ]:
analysis_pharm_df.info()

In [ ]:
analysis_pharm_df.describe()

In [ ]:
# Chỉ giữ lại các cột cần thiết cho mỗi khung dữ liệu
_chartevents_df = filtered_chartevents_df[['stay_id', 'subject_id', 'itemid', 'valuenum']]
_labs_df = filtered_labs_df[['subject_id', 'stay_id', 'itemid', 'value']]
_pharm_df = filtered_pharm_df[['subject_id', 'stay_id', 'drug']]

In [ ]:
# Thay thế các giá trị item_id bằng tên để dễ đọc hơn

d_items = pd.read_csv('/kaggle/input/data-icu/d_items.csv',low_memory=True)

d_items.head()

# Tạo một từ điển ánh xạ itemid, label từ chartevent_definitions
itemid_to_label = d_items.set_index('itemid')['label'].to_dict()


# thay thế mỗi giá trị itemid bằng giá trị tương ứng trong từ điển
_chartevents_df['itemid'] = _chartevents_df['itemid'].map(itemid_to_label)


d_labitems = pd.read_csv('/kaggle/input/data-icu/d_labitems.csv',low_memory=True)

# Tạo một từ điển ánh xạ itemid, label từ d_labitems
itemid_to_label = d_labitems.set_index('itemid')['label'].to_dict()


# thay thế mỗi giá trị itemid bằng giá trị tương ứng trong từ điển
_labs_df['itemid'] = _labs_df['itemid'].map(itemid_to_label)

In [ ]:
import pandas as pd

# Chuyển đổi cột 'valuenum' thành số, ép buộc các giá trị không phải số thành NaN
_chartevents_df['valuenum'] = pd.to_numeric(_chartevents_df['valuenum'], errors='coerce')

# Pivot DataFrame mà không có itemid ->  tạo ra một bảng tóm tắt
final_chartevents = _chartevents_df.pivot_table(
    index=['subject_id', 'stay_id'],
    columns='itemid',
    values='valuenum'
)

# Lặp lại các cột có giá trị bị thiếu và điền giá trị trung vi của mỗi 'itemid'
for col in final_chartevents.columns:
    median_value = final_chartevents[col].median()
    final_chartevents[col].fillna(median_value, inplace=True)


final_chartevents = final_chartevents.reset_index()


In [ ]:
import pandas as pd

# Chuyển đổi cột 'giá trị' thành số, ép buộc các giá trị không phải số thành NaN
_labs_df['value'] = pd.to_numeric(_labs_df['value'], errors='coerce')

# Pivot DataFrame mà không có itemid ->  tạo ra một bảng tóm tắt
final_labs = _labs_df.pivot_table(
    index=['subject_id', 'stay_id'],
    columns='itemid',
    values='value'
)

# Lặp lại các cột có giá trị bị thiếu và điền giá trị trung vi của mỗi 'itemid'
for col in final_labs.columns:
    median_value = final_labs[col].median()
    final_labs[col].fillna(median_value, inplace=True)

# Reset the index if needed
final_labs = final_labs.reset_index()


In [ ]:
# Tạo một cột mới 'giá trị' và đặt nó thành 1 cho tất cả các hàng
# thêm biến giả -> đánh dấu rằng một hàng tồn tại trong DataFrame
_pharm_df['value'] = 1

# Pivot 
final_pharm = _pharm_df.pivot_table(index=['subject_id', 'stay_id'], columns='drug', values='value', fill_value=0)


final_pharm = final_pharm.reset_index()
final_pharm.head(5)

In [ ]:
# Hợp nhất hai DataFrames đầu tiên 
icu_data = pd.merge(icustays_df, final_chartevents, on=['subject_id', 'stay_id'], how='inner')

# Hợp nhất 3  pivoted DataFrame
icu_data = pd.merge(icu_data, final_labs, on=['subject_id', 'stay_id'], how='inner')

#hợp nhất icustays_df DataFrame
icu_data = pd.merge(icu_data, final_pharm, on=['subject_id', 'stay_id'], how='inner')


icu_data.head(5)

In [ ]:
print(icu_data['stay_id'].nunique())

In [ ]:
# Từ điển bản đồ để đổi tên chủng tộc
race_mapping = {
    'ASIAN - ASIAN INDIAN': 'ASIAN',
    'ASIAN - CHINESE': 'ASIAN',
    'ASIAN - KOREAN': 'ASIAN',
    'ASIAN - SOUTH EAST ASIAN': 'ASIAN',
    'UNKNOWN': 'OTHER',
    'UNABLE TO OBTAIN': 'OTHER',
    'PATIENT DECLINED TO ANSWER': 'OTHER',
    'WHITE - RUSSIAN': 'WHITE',
    'PORTUGUESE': 'WHITE',
    'WHITE - BRAZILIAN': 'WHITE',
    'WHITE - OTHER EUROPEAN': 'WHITE',
    'WHITE - EASTERN EUROPEAN': 'WHITE', 
    'BLACK/CAPE VERDEAN':'BLACK/AFRICAN AMERICAN',
    'BLACK/AFRICAN':'BLACK/AFRICAN AMERICAN',
    'BLACK/CARIBBEAN ISLAND':'BLACK/AFRICAN AMERICAN',
    'HISPANIC/LATINO - CUBAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - PUERTO RICAN': 'HISPANIC/LATINO',
    'HISPANIC OR LATINO': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - SALVADORAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - DOMINICAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - HONDURAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - COLUMBIAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - GUATEMALAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - CENTRAL AMERICAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - MEXICAN': 'HISPANIC/LATINO',
    'SOUTH AMERICAN': 'HISPANIC/LATINO',
    'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'PACIFIC ISLANDER',
    'AMERICAN INDIAN/ALASKA NATIVE': 'PACIFIC ISLANDER',
    'NATIVE HAWAIIAN/PACIFIC ISLANDER': 'PACIFIC ISLANDER'
    
}

# Thay thế các giá trị trong cột 'race' bằng cách sử dụng từ điển ánh xạ cho tất cả các khung dữ liệu
icu_data['race'] = icu_data['race'].replace(race_mapping)
analysis_chartevents_df['race'] = analysis_chartevents_df['race'].replace(race_mapping)
analysis_labs_df['race'] = analysis_labs_df['race'].replace(race_mapping)
analysis_pharm_df['race'] = analysis_pharm_df['race'].replace(race_mapping)

# Hiển thị DataFrame đã cập nhật
print(analysis_chartevents_df['race'].unique().tolist())
print(analysis_labs_df['race'].unique().tolist())
print(analysis_pharm_df['race'].unique().tolist())

In [ ]:

print(icu_data.shape[0])
print(icu_data.shape[1])

In [ ]:
icu_data.to_csv('icu_data_Median.csv', index=False)

# Dataset Stadistics

In [ ]:
import pandas as pd

icu_data = pd.read_csv('icu_data_Median.csv',low_memory=True)
icu_stays = pd.read_csv('/kaggle/input/data-icu/icustays.csv', low_memory=True)
admissions = pd.read_csv('/kaggle/input/data-icu/admissions.csv',low_memory=True)
patients = pd.read_csv('/kaggle/input/data-icu/patients.csv', low_memory=True)


In [ ]:
icu_data.head()

In [ ]:
icu_data.columns

In [ ]:
icu_data.describe()

In [ ]:

# Hàm trả về phần trăm tùy thuộc vào một cột và điều kiện cụ thể

def percentage(df, column, condition):

    condition_number = len(df.loc[df[column]==condition])
    result = condition_number / len(df)
    
    return result

In [ ]:
icu_stays.columns

In [ ]:
 # lấy các giá trị duy nhất từ cột insurance
admissions['insurance'].unique()

In [ ]:
# tỷ lệ phần trăm của hàng Other 
result = percentage(admissions, 'insurance', 'Other')
print(result)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

labels = ['Other', 'Not Other']
sizes = [result * 100, (1 - result) * 100]
colors = ['#ff9999','#66b3ff']
explode = (0.1,0) 

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Tỷ lệ khung hình bằng nhau đảm bảo rằng hình tròn được vẽ theo dạng hình tròn.
plt.title("Percentage of 'Other' in Insurance Column")
plt.show()

In [ ]:
icu_data['hadm_id'].nunique()

In [ ]:
icu_stays['hadm_id'].loc[icu_stays['los']>1].nunique()

In [ ]:
len(icu_data[icu_data['age'] >= 18])

In [ ]:
result = len(icu_data[icu_data['los'] > 4])
print(result)

In [ ]:
analysis_labs_df.head(1)

# Data Visualizations for Disparities in Data

## Fig. 1: Depiction of Race breakdown -  - Miêu tả sự phân chia chủng tộc

###  (a) Average number of medications per ICU stay - Số lượng thuốc trung bình cho mỗi lần nằm ICU

In [ ]:
import matplotlib.pyplot as plt


# # Nhóm theo 'stay_id' và 'race' -> đếm số lượng sự kiện trong mỗi nhóm và lưu kết quả vào cột pharm_count
average_pharm = analysis_pharm_df.groupby(['stay_id', 'race']).size().reset_index(name='pharm_count').groupby('race')['pharm_count'].mean()

# Vẽ biểu đồ thanh(bar chart)
ax = average_pharm.plot(kind='bar', color='skyblue', edgecolor='black', figsize=(10,5))

# thêm nhãn vào thanh
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Average Number of Medications per first 24hr ICU Stay by Race')
plt.xlabel('Race')
plt.ylabel('Average Number of Medications Taken')
plt.show()

### (b) average number of labs per ICU stay - Số lượng xét nghiệm trung bình cho mỗi lần nằm ICU

In [ ]:
# analysis_labs_df

# Nhóm theo 'stay_id' và 'race' -> đếm số lượng sự kiện trong mỗi nhóm và lưu kết quả vào cột labs_count
average_labs = analysis_labs_df.groupby(['stay_id', 'race']).size().reset_index(name='labs_count').groupby('race')['labs_count'].mean()

# Vẽ biểu đồ thanh - bar chart
ax = average_labs.plot(kind='bar', color='skyblue', edgecolor='black', figsize=(10,5))

# thêm labels
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Average Number of Labs per first 24hr ICU Stay by Race')
plt.xlabel('Race')
plt.ylabel('Average Number of Labs Taken')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')
plt.show()

### (c) average number of vitals per ICU stay - số lượng trung bình các dấu hiệu sinh tồn trong mỗi lần nằm ICU

In [ ]:
# Nhóm theo 'stay_id' và 'race' -> đếm số lượng sự kiện trong mỗi nhóm và lưu kết quả vào cột chartevents_count
average_chartevents = analysis_chartevents_df.groupby(['stay_id', 'race']).size().reset_index(name='chartevents_count').groupby('race')['chartevents_count'].mean()

# Vẽ biểu đồ thanh - bar chart
ax = average_chartevents.plot(kind='bar', color='skyblue', edgecolor='black', figsize=(10,5))

# thêm labels
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Average Number of Vitals Taken per first 24hr ICU Stay by Race')
plt.xlabel('Race')
plt.ylabel('Average Number of Vitals Taken')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')
plt.show()

##  Fig. 2: Depiction of gender breakdown - - Miêu tả sự phân chia giới tính.

###  (a) The average number of medications per ICU stay -  Số lượng thuốc trung bình cho mỗi lần nằm ICU

In [ ]:


# Nhóm theo 'stay_id' và 'gender' -> đếm số lượng sự kiện trong mỗi nhóm và lưu kết quả vào cột pharm_count
average_pharm = analysis_pharm_df.groupby(['stay_id', 'gender']).size().reset_index(name='pharm_count').groupby('gender')['pharm_count'].mean()

# Biểu đồ thanh - bar chart
ax = average_pharm.plot(kind='bar', color='skyblue', edgecolor='black', figsize=(10,5))

# thêm labels
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Average Number of Medications Taken per first 24hr ICU Stay by Gender')
plt.xlabel('Gender')
plt.ylabel('Average Number of Medications Taken')
ax.set_xticklabels(['Male', 'Female'], rotation=90, ha='right')
plt.show()

### (b) the average number of labs per ICU stay - - số lượng xét nghiệm trung bình cho mỗi lần nằm ICU

In [ ]:
# Nhóm theo 'stay_id' và 'gender' -> đếm số lượng sự kiện trong mỗi nhóm và lưu kết quả vào cột labs_count
average_labs = analysis_labs_df.groupby(['stay_id', 'gender']).size().reset_index(name='labs_count').groupby('gender')['labs_count'].mean()

#  Biểu đồ bar chart
ax = average_labs.plot(kind='bar', color='skyblue', edgecolor='black', figsize=(10, 5))

# thêm nhãn
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Average Number of Labs per first 24hr ICU Stay by Gender')
plt.xlabel('Gender')
plt.ylabel('Average Number of Labs Taken')


ax.set_xticklabels(['Male', 'Female'], rotation=90, ha='right')

plt.show()

### (c) the average number of vitals per ICU stay - số lượng dấu hiệu sinh tồn trung bình trong mỗi lần nằm ICU

In [ ]:
# Nhóm theo 'stay_id' và 'gender' -> đếm số lượng sự kiện trong mỗi nhóm và lưu kết quả vào cột chartevents_count
average_chartevents = analysis_chartevents_df.groupby(['stay_id', 'gender']).size().reset_index(name='chartevents_count').groupby('gender')['chartevents_count'].mean()

# Biểu đồ thanh
ax = average_chartevents.plot(kind='bar', color='skyblue', edgecolor='black', figsize=(10,5))

# thêm labels
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Average Number of Vitals Taken per first 24hr ICU Stay by Gender')
plt.xlabel('Gender Grouping')
plt.ylabel('Average Number of Vitals Taken')
ax.set_xticklabels(['Male', 'Female'], rotation=90, ha='right')
plt.show()

##  Fig. 3: Depiction of insurance breakdown - Miêu tả về sự cố bảo hiểm

### (a) Average number of medications per ICU stay - Số lượng thuốc trung bình cho mỗi lần nằm ICU

In [ ]:
analysis_chartevents_df

In [ ]:

# Nhóm theo 'stay_id' và 'insurance' -> đếm số lượng sự kiện trong mỗi nhóm và lưu kết quả vào cột pharm_count
average_pharm = analysis_pharm_df.groupby(['stay_id', 'insurance']).size().reset_index(name='pharm_count').groupby('insurance')['pharm_count'].mean()

# Biểu đồ bar chart
ax = average_pharm.plot(kind='bar', color='skyblue', edgecolor='black', figsize=(10, 5))

# them labels
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Average Number of Medication given per first 24hr ICU Stay by Insurance')
plt.xlabel('Insurance')
plt.ylabel('Average Number of Medications Given')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')
plt.show()

### (b) average number of labs per ICU stay -  số lượng xét nghiệm trung bình cho mỗi lần nằm ICU

In [ ]:
# Nhóm theo 'stay_id' và 'insurance' -> đếm số lượng sự kiện trong mỗi nhóm và lưu kết quả vào cột labs_count
average_labs = analysis_labs_df.groupby(['stay_id', 'insurance']).size().reset_index(name='labs_count').groupby('insurance')['labs_count'].mean()

# Biểu đồ thanh
ax = average_labs.plot(kind='bar', color='skyblue', edgecolor='black', figsize=(10, 5))

# thêm nhãn
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Average Number of Labs per first 24hr ICU Stay by Insurance')
plt.xlabel('Insurance')
plt.ylabel('Average Number of Labs Taken')
# Rotate x-axis labels
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')
plt.show()

### (c) average number of vitals per ICU stay - số lượng trung bình các dấu hiệu sinh tồn trong mỗi lần nằm ICU

In [ ]:
# Nhóm theo 'stay_id' và 'insurance' -> đếm số lượng sự kiện trong mỗi nhóm và lưu kết quả vào cột chartevents_count
average_chartevents = analysis_chartevents_df.groupby(['stay_id', 'insurance']).size().reset_index(name='chartevents_count').groupby('insurance')['chartevents_count'].mean()

# Biểu đồ thanh
ax = average_chartevents.plot(kind='bar', color='skyblue', edgecolor='black', figsize=(10, 5))

# them nhan
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Average Number of Vitals Taken per first 24hr ICU Stay by Insurance')
plt.xlabel('Insurance')
plt.ylabel('Average Number of Vitals Taken')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')
plt.show()

## Fig. 4: Length of ICU Stay by Insurance and Race - Thời gian nằm ICU theo Bảo hiểm và Chủng tộc

In [ ]:
import seaborn as sns
# icu_data

# Đặt kiểu seaborn thành whitegrid
sns.set_style("whitegrid")

# Vẽ biểu đồ thanh nhóm cho LOS trung bình
plt.figure(figsize=(20, 8))
ax = sns.barplot(x='race', y='los', hue='insurance', data=icu_data, palette='colorblind', estimator=np.mean)

plt.title('Average Length of Stay (LOS) by Insurance and Race', fontsize = 25)
plt.xlabel('Race',fontsize = 18)
plt.ylabel('Average Length of Stay (LOS)', fontsize = 18)

plt.legend(title='Insurance', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# Đặt kiểu seaborn thành whitegrid
sns.set_style("whitegrid")

# Vẽ biểu đồ thanh nhóm cho LOS trung bình cho insurance
plt.figure(figsize=(14, 8))
ax = sns.barplot(x='insurance', y='los', hue='race', data=icu_data, palette='colorblind', estimator=np.mean)

plt.title('Average Length of Stay (LOS) by Insurance and Race', fontsize = 25)
plt.xlabel('Race', fontsize = 18)
plt.ylabel('Average Length of Stay (LOS)', fontsize = 18)

plt.legend(title='Race', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

# ANOVA ANALYSIS

In [ ]:
# tính trung bình thời gian nằm viện (los) cho mỗi nhóm
grouped_data = icu_data.groupby(['gender', 'insurance', 'race'])['los'].mean()


print(grouped_data)

In [ ]:
grouped_data = grouped_data.reset_index()

# Tạo bảng pivot để dễ dàng trực quan hóa
pivot_table = grouped_data.pivot_table(index=['gender', 'insurance'], columns='race', values='los')

# Vẽ bảng pivot dưới dạng biểu đồ nhiệt
plt.figure(figsize=(12, 8))
plt.title('Average Length of Stay (LOS) by Gender, Insurance, and Race')
sns.heatmap(pivot_table, annot=True, cmap='viridis', fmt=".2f")
plt.xlabel('Race', fontsize=16)
plt.ylabel('Gender and Insurance', fontsize=16)
plt.show()

In [ ]:
from scipy.stats import f_oneway
# icu_data

# Thực hiện kiểm tra ANOVA một chiều cho 'gender'
anova_gender = f_oneway(*[group['los'].values for name, group in icu_data.groupby('gender')])

# Thực hiện kiểm tra ANOVA một chiều cho 'insurance'
anova_insurance = f_oneway(*[group['los'].values for name, group in icu_data.groupby('insurance')])

# Thực hiện kiểm tra ANOVA một chiều cho 'race'
anova_race = f_oneway(*[group['los'].values for name, group in icu_data.groupby('race')])

# in ra ket qua
print("ANOVA for Gender:")
print(anova_gender)

print("\nANOVA for Insurance:")
print(anova_insurance)

print("\nANOVA for Race:")
print(anova_race)


In [ ]:
anova_results = {
    'Factor': ['Gender', 'Insurance', 'Race'],
    'F-Statistic': [
        anova_gender.statistic,
        anova_insurance.statistic,
        anova_race.statistic
    ],
    'p-Value': [
        anova_gender.pvalue,
        anova_insurance.pvalue,
        anova_race.pvalue
    ]
}

# Create a DataFrame from the dictionary
anova_summary = pd.DataFrame(anova_results)

# Print the summary table
print("\nANOVA Summary Table:")
anova_summary

# Model XGBOOST - Replication

In [ ]:
condition = (icu_data['los'] > 4).map({True: 'Long', False: 'Short'})

# Tạo một cột mới 'los_type' dựa trên điều kiện
icu_data['los_type'] = condition

In [ ]:
# Chia dữ liệu thành dữ liệu training  và dữ liệu validation
from sklearn.model_selection import train_test_split

# Giả sử 'subject_id' là mã định danh bệnh nhân
patient_ids = icu_data['subject_id'].unique()

# Chia patient_ids thành các tập huấn luyện và thử nghiệm
train_patient_ids, validation_patient_ids = train_test_split(patient_ids, test_size=0.2, random_state=42)

# Lọc khung dữ liệu dựa trên sự phân chia của patient_ids
train_data = icu_data[icu_data['subject_id'].isin(train_patient_ids)]
validation_data = icu_data[icu_data['subject_id'].isin(validation_patient_ids)]

# Kiểm tra hình dạng của tập huấn luyện và tập kiểm tra
print("Training set shape:", train_data.shape)
print("Validation set shape:", validation_data.shape)

In [ ]:
# Đếm số lượng các giá trị khác nhau trong cột 'insurance' ở tập train
train_insurance_counts =train_data['insurance'].value_counts()
print("\033[1mTrain Data Insurance Counts\033[0m:")
print(train_insurance_counts)
print()

# Đếm số lượng các giá trị khác nhau trong cột 'insurance' ở test
validation_insurance_counts =validation_data['insurance'].value_counts()
print("\033[1mValidation Data Insurance Counts:\033[0m")
print(validation_insurance_counts)
print()

# Đếm số lượng các giá trị khác nhau trong cột 'race'
train_race_counts =train_data['race'].value_counts()
print("\033[1mTrain Data Race Counts:\033[0m")
print(train_race_counts)
print()

# Đếm số lượng các giá trị khác nhau trong cột 'race'
validation_race_counts =validation_data['race'].value_counts()
print("\033[1mValidation Data Race Counts:\033[0m")
print(validation_race_counts)
print()

# Đếm số lượng các giá trị khác nhau trong cột 'los_type'
train_duration_counts =train_data['los_type'].value_counts()
print("\033[1mTrain Data LOS Counts:\033[0m")
print(train_duration_counts)
print()

# Đếm số lượng các giá trị khác nhau trong cột 'los_type'
validation_duration_counts =validation_data['los_type'].value_counts()
print("\033[1mValidation Data LOS Counts:\033[0m")
print(validation_duration_counts)

In [ ]:
# lấy cột subject_id duy nhất
patient_ids_train = train_data['subject_id'].unique()

# Chia patient_ids_train thành các tập huấn luyện và thử nghiệm
train_train_ids, test_train_ids = train_test_split(patient_ids_train, test_size=0.2, random_state=42)

# Lọc train_data dựa trên sự phân chia của patient_ids_train
train_train_data = train_data[train_data['subject_id'].isin(train_train_ids)]
test_train_data = train_data[train_data['subject_id'].isin(test_train_ids)]

# Kiểm tra hình dạng của tập huấn luyện và tập kiểm tra
print("Train Training set shape:", train_train_data.shape)
print("Train Testing set shape:", test_train_data.shape)

# 1, XGBoost trên tập train_train_data và test_train_data

In [ ]:
train_train_data

In [ ]:
test_train_data

In [ ]:
!pip install xgboost
import pandas as pd
import xgboost
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Các cột cần loại trừ khỏi các tính năng
exclude_columns = ['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los', 'los_type', 
                   'age', 'gender', 'insurance', 'race', 'PHYSICIAN REFERRAL']

# Xác định các cột tính năng của bạn (loại trừ các cột nhất định)
feature_columns = [col for col in train_data.columns if col not in exclude_columns]

# Loại trừ các cột không phải số trước khi chia tỷ lệ
numeric_columns = train_train_data[feature_columns].select_dtypes(include=['float64', 'int64']).columns
X_train = train_train_data[numeric_columns]
y_train = train_train_data['los_type']
X_test = test_train_data[numeric_columns]
y_test = test_train_data['los_type']

# Chuyển đổi 'Short' và 'Long' thành số (0 và 1)
y_train = y_train.map({'Short': 0, 'Long': 1})
y_test = y_test.map({'Short': 0, 'Long': 1})

# Chuẩn hóa các tính năng nếu cần (tùy chọn nhưng thường có lợi)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#  Tạo  XGBoost classifier
model = XGBClassifier(objective='binary:logistic', random_state=42)

# Huấn luyện mô hình
model.fit(X_train, y_train)

# Đưa ra dự đoán trên tập kiểm tra
y_pred_prob = model.predict_proba(X_test)[:, 1]

# Chuyển đổi điểm xác suất thành dự đoán nhị phân bằng cách sử dụng ngưỡng
threshold = 0.5
y_pred = (y_pred_prob >= threshold).astype(int)

# Tạo đường cong ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Tạo đường cong độ chính xác-thu hồi
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
average_precision = average_precision_score(y_test, y_pred_prob)


In [ ]:
# Vẽ đường cong ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

# Vẽ đường cong độ chính xác-thu hồi
plt.figure(figsize=(8, 6))
plt.step(recall, precision, color='b', where='post', label='Precision-Recall curve (AP = {:.2f})'.format(average_precision))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='upper right')
plt.show()
print(f'Area under ROC curve (AUC-ROC): {roc_auc:.2f}')
print(f'Area under Precision-Recall curve (AUC-PR): {average_precision:.2f}')

# Tạo ma trận nhầm lẫn và báo cáo phân loại
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

print("Confusion Matrix:")
print(cm)

print("\nClassification Report:")
print(cr)


## 1.1, Insurance - Bảo hiểm

In [ ]:
# Các cột cần loại trừ khỏi các tính năng
exclude_columns = ['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los', 'los_type', 
                   'age', 'gender', 'insurance', 'race', 'PHYSICIAN REFERRAL']

# Xác định các cột tính năng của bạn (loại trừ các cột nhất định)
feature_columns = [col for col in train_data.columns if col not in exclude_columns]

# Loại trừ các cột không phải số trước khi chia tỷ lệ
numeric_columns = train_train_data[feature_columns].select_dtypes(include=['float64', 'int64']).columns
X_train = train_train_data[numeric_columns]
y_train = train_train_data['los_type']
X_test = test_train_data[numeric_columns]
y_test = test_train_data['los_type']


# Chuyển đổi 'Short' và 'Long' thành số (0 và 1)
y_train = y_train.map({'Short': 0, 'Long': 1})
y_test = y_test.map({'Short': 0, 'Long': 1})

# Chuẩn hóa các feature nếu cần (tùy chọn nhưng thường có lợi)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# # Trích xuất thông tin bảo hiểm cho test set
insurance_test = train_data.loc[y_test.index, 'insurance']

# # Tạo XGBoost 
model = XGBClassifier(objective='binary:logistic', random_state=42)

# Huấn luyện model
model.fit(X_train, y_train)

# Đưa ra dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

# Kết hợp thông tin dự đoán và bảo hiểm vào một DataFrame
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Insurance': insurance_test})

# Đánh giá hiệu suất cho từng loại bảo hiểm
insurance_types = results_df['Insurance'].unique()

for insurance_type in insurance_types:
    subset = results_df[results_df['Insurance'] == insurance_type]
    
    # Tạo ma trận nhầm lẫn và báo cáo phân loại
    cm = confusion_matrix(subset['Actual'], subset['Predicted'])
    cr = classification_report(subset['Actual'], subset['Predicted'])
    
    print(f"Results for {insurance_type} insurance:")
    print("Confusion Matrix:")
    print(cm)
    print("\nClassification Report:")
    print(cr)
    print('\n' + '-'*50 + '\n')

## 1.2, Race - chủng tộc

In [ ]:
# Các cột cần loại trừ khỏi các tính năng
exclude_columns = ['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los', 'los_type', 
                   'age', 'gender', 'insurance', 'race', 'PHYSICIAN REFERRAL']

# Xác định các cột tính năng của bạn (loại trừ các cột nhất định)
feature_columns = [col for col in train_data.columns if col not in exclude_columns]

# Loại trừ các cột không phải số trước khi chia tỷ lệ
numeric_columns = train_train_data[feature_columns].select_dtypes(include=['float64', 'int64']).columns
X_train = train_train_data[numeric_columns]
y_train = train_train_data['los_type']
X_test = test_train_data[numeric_columns]
y_test = test_train_data['los_type']

# Chuyển đổi 'Short' và 'Long' thành nhãn số (0 và 1)
y_train = y_train.map({'Short': 0, 'Long': 1})
y_test = y_test.map({'Short': 0, 'Long': 1})

# Chuẩn hóa các tính năng nếu cần (tùy chọn nhưng thường có lợi)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Trích xuất thông tin chủng tộc cho bộ kiểm tra
race_test = train_data.loc[y_test.index, 'race']

# tạo XGBoost classifier
model = XGBClassifier(objective='binary:logistic', random_state=42)

# Huấn luyện model
model.fit(X_train, y_train)

# Đưa ra dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

#  Kết hợp thông tin dự đoán và chủng tộc vào một DataFrame
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Race': race_test})

# Tạo bảng chứa cột chủng tộc duy nhất 
race_types = results_df['Race'].unique()

# Đánh giá hiệu suất cho từng loại chủng tộc
for race_type in race_types:
    subset = results_df[results_df['Race'] == race_type]
    
    # Tạo ma trận nhầm lẫn và báo cáo phân loại
    cm = confusion_matrix(subset['Actual'], subset['Predicted'])
    cr = classification_report(subset['Actual'], subset['Predicted'])
    
    print(f"Results for {race_type} race:")
    print("Confusion Matrix:")
    print(cm)
    print("\nClassification Report:")
    print(cr)
    print('\n' + '-'*50 + '\n')


# Đánh giá sự công bằng trong các mô hình học máy

In [ ]:
#  tính toán sự khác biệt về tỷ lệ dương tính thật 
# (True Positive Rate - TPR) và tỷ lệ dương tính giả (False Positive Rate - FPR)
def equalized_odds(y_true, y_pred, sensitive_feature):
    # Tính toán ma trận nhầm lẫn
    cm_protected = confusion_matrix(y_true[sensitive_feature], y_pred[sensitive_feature])
    cm_non_protected = confusion_matrix(y_true[~sensitive_feature], y_pred[~sensitive_feature])

    # Tính tỷ lệ dương tính thật (độ nhạy) và tỷ lệ dương tính giả (rò rỉ) cho cả hai nhóm
    tpr_protected = cm_protected[1, 1] / cm_protected.sum(axis=1)[1]
    fpr_protected = cm_protected[0, 1] / cm_protected.sum(axis=1)[0]

    tpr_non_protected = cm_non_protected[1, 1] / cm_non_protected.sum(axis=1)[1]
    fpr_non_protected = cm_non_protected[0, 1] / cm_non_protected.sum(axis=1)[0]

    # Tính toán sự khác biệt tuyệt đối về TPR và FPR giữa nhóm được bảo vệ và nhóm không được bảo vệ
    tpr_difference = abs(tpr_protected - tpr_non_protected)
    fpr_difference = abs(fpr_protected - fpr_non_protected)

    return tpr_difference + fpr_difference

# Tính toán sự khác biệt về tỷ lệ dự đoán dương tính
def demographic_parity(y_pred, sensitive_feature):
    proportion_protected = y_pred[sensitive_feature].mean()
    proportion_non_protected = y_pred[~sensitive_feature].mean()

    return abs(proportion_protected - proportion_non_protected)

# 2, XGBoost trên tập validation

In [ ]:
# validation_data = icu_data[icu_data['subject_id'].isin(validation_patient_ids)]
validation_data

In [ ]:
# Các cột cần loại trừ khỏi các tính năng
exclude_columns = ['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los', 'los_type', 
                   'age', 'gender', 'insurance', 'race', 'PHYSICIAN REFERRAL']

# Xác định các cột tính năng của bạn (loại trừ các cột nhất định)
feature_columns = [col for col in validation_data.columns if col not in exclude_columns]

# Loại trừ các cột không phải số trước khi chia tỷ lệ
numeric_columns = validation_data[feature_columns].select_dtypes(include=['float64', 'int64']).columns
X_verify = validation_data[numeric_columns]
y_verify = validation_data['los_type']

# Chuyển đổi 'Short' và 'Long' thành nhãn số (0 và 1)
y_verify = y_verify.map({'Short': 0, 'Long': 1})

# Chuẩn hóa các tính năng bằng cách sử dụng cùng một bộ chia tỷ lệ từ tập huấn luyện
X_verify = scaler.transform(X_verify)

# Đưa ra dự đoán về tập xác minh bằng cách sử dụng mô hình hiện có
y_verify_pred_prob = model.predict_proba(X_verify)[:, 1]

# Chuyển đổi điểm xác suất thành dự đoán nhị phân bằng cách sử dụng ngưỡng
y_verify_pred = (y_verify_pred_prob > threshold).astype(int)

# Tạo đường cong ROC cho tập xác minh
fpr_verify, tpr_verify, thresholds_verify = roc_curve(y_verify, y_verify_pred_prob)
roc_auc_verify = auc(fpr_verify, tpr_verify)

# Tạo đường cong độ chính xác-thu hồi cho tập xác minh
precision_verify, recall_verify, _ = precision_recall_curve(y_verify, y_verify_pred_prob)
average_precision_verify = average_precision_score(y_verify, y_verify_pred_prob)

In [ ]:
# Vẽ đường cong ROC cho tập xác minh
plt.figure(figsize=(8, 6))
plt.plot(fpr_verify, tpr_verify, color='darkorange', lw=2, label='ROC curve (AUC = {:.2f})'.format(roc_auc_verify))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve - Validation Set')
plt.legend(loc='lower right')
plt.show()

# Vẽ đường cong độ chính xác-thu hồi cho tập xác minh
plt.figure(figsize=(8, 6))
plt.step(recall_verify, precision_verify, color='b', where='post', label='Precision-Recall curve (AP = {:.2f})'.format(average_precision_verify))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - Validation Set')
plt.legend(loc='upper right')
plt.show()

print(f'Area under ROC curve (AUC-ROC) for Verification Set: {roc_auc_verify:.2f}')
print(f'Area under Precision-Recall curve (AUC-PR) for Verification Set: {average_precision_verify:.2f}')


# Tạo ma trận nhầm lẫn và báo cáo phân loại cho tập xác minh
cm_verify = confusion_matrix(y_verify, y_verify_pred)
cr_verify = classification_report(y_verify, y_verify_pred)

print("Confusion Matrix for Test Set:")
print(cm_verify)

print("\nClassification Report for Test Set:")
print(cr_verify)

In [ ]:
print("\nClassification Report for Test Set:")
print(cr_verify)

In [ ]:
print("Confusion Matrix for Test Set:")
cm_verify

In [ ]:
import numpy as np
y_true = validation_data['los_type'].map({'Short': 0, 'Long': 1})

# 'validation_data' có các dự đoán được lưu trữ trong 'y_verify_pred'
cm_verify = confusion_matrix(y_true, y_verify_pred)

# Trích xuất các giá trị từ ma trận nhầm lẫn
tn, fp, fn, tp = cm_verify.ravel()

#  Tính toán tỷ lệ cược cân bằng
equalized_odds_race = (tp / (tp + fn)) / (fp / (fp + tn))

print(equalized_odds_race)

## Insurance

In [ ]:

exclude_columns = ['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los', 'los_type', 
                   'age', 'gender', 'race', 'PHYSICIAN REFERRAL']


feature_columns = [col for col in validation_data.columns if col not in exclude_columns]

# Loại trừ các cột không phải số trước khi chia tỷ lệ
numeric_columns = validation_data[feature_columns].select_dtypes(include=['float64', 'int64']).columns
X = validation_data[numeric_columns]
y = validation_data['los_type']


y = y.map({'Short': 0, 'Long': 1})


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Trích xuất thông tin bảo hiểm cho bộ kiểm tra
insurance_test = validation_data.loc[y_test.index, 'insurance']


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


model = XGBClassifier(objective='binary:logistic', random_state=42)


model.fit(X_train, y_train)


y_pred = model.predict(X_test)


results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Insurance': insurance_test})

# Đánh giá hiệu suất cho từng loại bảo hiểm
insurance_types = results_df['Insurance'].unique()

for insurance_type in insurance_types:
    subset = results_df[results_df['Insurance'] == insurance_type]
    
    # Tạo ma trận nhầm lẫn và báo cáo phân loại
    cm = confusion_matrix(subset['Actual'], subset['Predicted'])
    cr = classification_report(subset['Actual'], subset['Predicted'])
    
    print(f"Results for {insurance_type} insurance:")
    print("Confusion Matrix:")
    print(cm)
    print("\nClassification Report:")
    print(cr)
    print('\n' + '-'*50 + '\n')

## Race

In [ ]:

exclude_columns = ['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los', 'los_type', 
                   'age', 'gender', 'race', 'PHYSICIAN REFERRAL']


feature_columns = [col for col in validation_data.columns if col not in exclude_columns]

# Loại trừ các cột không phải số trước khi chia tỷ lệ
numeric_columns = validation_data[feature_columns].select_dtypes(include=['float64', 'int64']).columns
X = validation_data[numeric_columns]
y = validation_data['los_type']

# Chuyển đổi 'Ngắn' và 'Dài' thành nhãn số (0 và 1)
y = y.map({'Short': 0, 'Long': 1})


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


race_test = validation_data.loc[y_test.index, 'race']


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


model = XGBClassifier(objective='binary:logistic', random_state=42)


model.fit(X_train, y_train)

y_pred = model.predict(X_test)


results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Race': race_test})

# Đánh giá hiệu suất cho từng loại
for race_type in race_types:
    subset = results_df[results_df['Race'] == race_type]
    
    # Tạo ma trận nhầm lẫn và báo cáo phân loại
    cm = confusion_matrix(subset['Actual'], subset['Predicted'])
    cr = classification_report(subset['Actual'], subset['Predicted'])
    
    print(f"Results for {race_type} race:")
    print("Confusion Matrix:")
    print(cm)
    print("\nClassification Report:")
    print(cr)
    print('\n' + '-'*50 + '\n')